# REI505M Machine Learning - Final project
### Due: --------

**Names**: Axel Kristján Axelsson, Bjarni Haukur Bjarnason <br />
**Email**: aka30@hi.is, bhb23@hi.is

**1. Image classification**

s


In [8]:
from data_loader import MyDataLoader
%load_ext autoreload 
%autoreload 2
dl = MyDataLoader("combine2", (128,128))



The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
dl.normalize_train_data(norm_size=(128,128))

100%|██████████| 102/102 [00:54<00:00,  1.89it/s]


In [9]:
try:
    del X
    del y

except:
    pass

X, y = dl.get_lab_data()

100%|██████████| 102/102 [00:55<00:00,  1.85it/s]


In [16]:
import numpy as np

print(X.shape)
print(y.shape)
print(np.max(X))
print(np.min(y))

(11636, 128, 128, 1)
(11636, 128, 128, 2)
100
-107


In [4]:
X_lab = lab[:,:,:,0]/100
y_lab = lab[:,:,:,1:]/128

In [5]:
import numpy as np

print(np.max(X_lab))
print(np.max(y_lab))
print(np.min(X_lab))
print(np.min(y_lab))

1.0
0.7648338107997892
0.0
-0.8414919384200387


In [6]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_lab, y_lab, test_size = 0.3, random_state = 3)
X_test, X_val, y_test, y_val = train_test_split(X_test, y_test, test_size = 0.33, random_state = 3)

In [7]:
import numpy as np

X_bla = np.expand_dims(X_train, -1)
y_bla = y_train
print(y_bla.shape)
print(np.max(X_bla))
print(np.max(y_bla))
print(np.min(X_bla))
print(np.min(y_bla))


(8145, 128, 128, 2)
1.0
0.7648338107997892
0.0
-0.8272612483944879


In [17]:
import tensorflow as tf
tf.keras.backend.clear_session()

In [9]:

from keras.layers import Input, Conv2D, BatchNormalization, UpSampling2D, Dense, InputLayer
from keras.models import Model
from keras.regularizers import l2
from tensorflow.keras.optimizers import Adam
from keras import Sequential




img_shape = (128, 128, 1)
kernel_size = (3,3)
num_filters = 64

try:
    del model
except:
    pass

model = Sequential()
model.add(InputLayer(input_shape=img_shape))
model.add(Conv2D(num_filters, kernel_size, activation='relu', padding='same'))
model.add(Conv2D(num_filters, kernel_size, activation='relu', padding='same', strides=2))
model.add(Conv2D(num_filters*2, kernel_size, activation='relu', padding='same'))
model.add(Conv2D(num_filters*2, kernel_size, activation='relu', padding='same', strides=2))
model.add(Conv2D(num_filters*4, kernel_size, activation='relu', padding='same'))
model.add(Conv2D(num_filters*4, kernel_size, activation='relu', padding='same', strides=2))
model.add(Conv2D(num_filters*8, kernel_size, activation='relu', padding='same'))
model.add(Conv2D(num_filters*4, kernel_size, activation='relu', padding='same'))
model.add(Conv2D(num_filters*2, kernel_size, activation='relu', padding='same'))
model.add(UpSampling2D((2, 2)))
model.add(Conv2D(num_filters, kernel_size, activation='relu', padding='same'))
model.add(UpSampling2D((2, 2)))
model.add(Conv2D(32, kernel_size, activation='relu', padding='same'))
model.add(Conv2D(2, kernel_size, activation='softmax', padding='same'))
model.add(UpSampling2D((2, 2)))



model.compile(optimizer='rmsprop', loss='mse')

model.fit(X_bla, y_bla, batch_size = 32, epochs = 30)

Epoch 1/30
255/255 [==============================] - 17s 63ms/step - loss: 0.2162
Epoch 2/30
255/255 [==============================] - 16s 62ms/step - loss: 0.2149
Epoch 3/30
255/255 [==============================] - 16s 63ms/step - loss: 0.2147
Epoch 4/30
255/255 [==============================] - 16s 62ms/step - loss: 0.2145
Epoch 5/30
255/255 [==============================] - 13s 52ms/step - loss: 0.2142
Epoch 6/30
255/255 [==============================] - 13s 50ms/step - loss: 0.2140
Epoch 7/30
255/255 [==============================] - 13s 50ms/step - loss: 0.2138
Epoch 8/30
255/255 [==============================] - 13s 50ms/step - loss: 0.2136
Epoch 9/30
255/255 [==============================] - 13s 50ms/step - loss: 0.2134
Epoch 10/30
255/255 [==============================] - 13s 51ms/step - loss: 0.2133
Epoch 11/30
255/255 [==============================] - 13s 50ms/step - loss: 0.2130
Epoch 12/30
255/255 [==============================] - 13s 51ms/step - loss: 0.2128
E

In [13]:
# model.fit(X_train, y_train, batch_size = 32, epochs = 100, validation_data=(X_val, y_val))
pred = model.predict(np.expand_dims(np.divide(X_test[:100],100),-1))
pred = pred*128
print(np.max(pred[1]))
print(np.min(pred[1]))


65.87507
62.124935


In [14]:

mynd = pred[1]
fokking = np.zeros((128,128,3))
print(X_test[1].shape)
fokking[:,:,0] = X_test[1]
fokking[:,:,1:] = mynd
fokking = fokking.astype(int)
print(np.max(fokking))
print(np.min(fokking))

(128, 128)
65
0


In [15]:
drasl = color.lab2rgb(fokking)
print(np.max(drasl))
print(np.min(drasl))

4.501204800696596e-10
0.0


In [16]:
from PIL import Image

img = Image.fromarray(drasl)

imgO = Image.fromarray(X_test[1])
# imgTrue = Image.fromarray(y_test[1], 'RGB')
# img = Image.fromarray(mynd1, 'RGB')
imgO.show()
# imgTrue.show()
img.show()

TypeError: Cannot handle this data type: (1, 1, 3), <f8

In [18]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))


Num GPUs Available:  1


In [22]:
print(X.shape)
print(X[11001])


(11636, 500, 500)
[[184 184 184 ... 187 187 187]
 [184 184 184 ... 187 187 187]
 [184 184 184 ... 187 187 187]
 ...
 [169 167 166 ... 209 209 209]
 [171 169 167 ... 209 209 209]
 [172 170 168 ... 209 209 209]]


In [7]:


# print(X.shape)
# print(X_test.shape)
# im = Image.fromarray(X_train[0], mode="YCbCr")
# im.show()
# print(X_test[0])

(9262, 500, 500, 3)
(9262, 500, 500, 3)
[[[ 55  45  35]
  [ 56  46  36]
  [ 56  46  36]
  ...
  [ 47  43  31]
  [ 47  43  31]
  [ 47  43  31]]

 [[ 52  42  32]
  [ 53  43  33]
  [ 54  44  34]
  ...
  [ 52  48  36]
  [ 52  48  36]
  [ 52  48  36]]

 [[ 49  39  29]
  [ 50  40  30]
  [ 51  41  31]
  ...
  [ 55  51  39]
  [ 55  51  39]
  [ 55  51  39]]

 ...

 [[146 106 107]
  [146 106 107]
  [145 105 105]
  ...
  [165 158 166]
  [165 158 166]
  [165 158 166]]

 [[158 118 119]
  [158 118 119]
  [157 117 117]
  ...
  [175 168 176]
  [175 168 176]
  [175 168 176]]

 [[165 125 126]
  [164 124 125]
  [164 124 124]
  ...
  [180 173 181]
  [180 173 181]
  [180 173 181]]]


In [8]:
from PIL import Image
import numpy as np

img = Image.open("test.jpg").convert('L')
img = img.resize((500,500))
img.show()
arr = np.array(img)

In [18]:
from PIL import Image
import numpy as np

img = Image.open("bla.jpg")
img = img.resize((500,500))
x=np.ndarray((500,500,3), dtype=np.short)
img0 = Image.fromarray(x)

TypeError: Cannot handle this data type: (1, 1, 3), <i2